# Naive Bayes - Implementação

In [1]:
import pandas as pd

df = pd.read_csv('golf.csv',sep='\t')
print(df.shape)

(14, 5)


In [2]:
df.describe()

,Clima,Temperatura,Umidade,Vento,Jogar Golf?
count,14,14,14,14,14
unique,3,3,2,2,2
top,Chuvoso,Moderado,Alta,FALSO,Sim
freq,5,6,7,8,9


## Qual a probabilidade de jogar golf?

In [3]:
df_sim = df[df['Jogar Golf?'] == 'Sim']
df_nao = df[df['Jogar Golf?'] == 'Não']

total_sim = len(df_sim)
total_nao = len(df_nao)
print("Total de sim na coluna:", total_sim)
print("Total de não na coluna:", total_nao)

Total de sim na coluna: 9
Total de não na coluna: 5


## Encontrando a probabilidade dos valores para o atributo Clima

In [4]:
# vamos armazenar as probabilidades para cada valor do clima referente à Jogar Golf = Sim
clima = dict()

# quais valores esse atributo pode assumir?
df['Clima'].unique()

array(['Chuvoso', 'Nublado', 'Ensolarado'], dtype=object)

In [5]:
valores = df['Clima'].unique()

# pra cada valor, vamos ver quantas linhas tem com a classe sim

for v in valores:
    linhas = df[ (df['Clima'] == v) & (df['Jogar Golf?'] == 'Sim') ]
    print(v, len(linhas))

Chuvoso 2
Nublado 4
Ensolarado 3


In [6]:
# para calcular a probabilidade, precisamos dividir essas quantidades pelo total de Jogar Golf? = Sim da base

for v in valores:
    linhas = df[ (df['Clima'] == v) & (df['Jogar Golf?'] == 'Sim') ]
    print(v, len(linhas)/total_sim)


Chuvoso 0.2222222222222222
Nublado 0.4444444444444444
Ensolarado 0.3333333333333333


In [7]:
# vamos fazer isso para a classe Jogar Golf = Não agora

for v in valores:
    linhas = df[ (df['Clima'] == v) & (df['Jogar Golf?'] == 'Não') ]
    print(v, len(linhas)/total_nao)

Chuvoso 0.6
Nublado 0.0
Ensolarado 0.4


In [8]:
# sempre que uma probabilidade estiver zerada, precisamos corrigir ela com uma pequena adição pra não dar erro de cálculo

for v in valores:
    linhas = df[ (df['Clima'] == v) & (df['Jogar Golf?'] == 'Não') ]
    p = len(linhas)/total_nao
    if p == 0.0:
        p += 0.001
    print(v, p)

Chuvoso 0.6
Nublado 0.001
Ensolarado 0.4


## Calculando a probabilidade de cada atributo para cada classe

In [9]:
prob_sim = dict()
prob_nao = dict()

colunas = list(df.columns)
colunas.remove('Jogar Golf?')
print(colunas)

['Clima', 'Temperatura', 'Umidade', 'Vento']


In [10]:
for c in colunas:
    prob_sim[c] = dict()
    prob_nao[c] = dict()

    valores_possiveis = df[c]

    valores = list(df_sim[c])
    total = len(valores)
    for v in set(valores_possiveis):
        prob_sim[c][v] = valores.count(v) / total
        if prob_sim[c][v] == 0:
            prob_sim[c][v] = 0.0001

    valores = list(df_nao[c])
    total = len(valores)
    for v in set(valores_possiveis):
        prob_nao[c][v] = valores.count(v) / total
        if prob_nao[c][v] == 0:
            prob_nao[c][v] = 0.0001

In [11]:
prob_nao['Clima'] # P(clima=chuvoso|JogarGolf=Não)

{'Chuvoso': 0.6, 'Ensolarado': 0.4, 'Nublado': 0.0001}

## Classificar a amostra Clima = Chuvoso, Temperatura = Frio, Umidade = Alta e Vento = Verdadeiro

In [12]:
nao = prob_nao['Clima']['Chuvoso'] * prob_nao['Temperatura']['Frio'] * prob_nao['Umidade']['Alta'] * prob_nao['Vento']['VERDADEIRO'] * (total_nao/total)
sim = prob_sim['Clima']['Chuvoso'] * prob_sim['Temperatura']['Frio'] * prob_sim['Umidade']['Alta'] * prob_sim['Vento']['VERDADEIRO'] * (total_sim/total)

In [13]:
if sim > nao:
    print("Probabilidade de que as pessoas vão jogar golf.")
else:
    print("Probabilidade de que as pessoas não vão jogar golf.")

Probabilidade de que as pessoas não vão jogar golf.


In [14]:
print(sim)
print(nao)

0.014814814814814814
0.0576


In [15]:
## E se eu levar em consideração apenas o clima?

print(prob_sim['Clima'])
print(prob_nao['Clima'])

{'Chuvoso': 0.2222222222222222, 'Ensolarado': 0.3333333333333333, 'Nublado': 0.4444444444444444}
{'Chuvoso': 0.6, 'Ensolarado': 0.4, 'Nublado': 0.0001}


In [16]:
## E se eu levar em consideração apenas a temperatura?

print(prob_sim['Temperatura'])
print(prob_nao['Temperatura'])

{'Quente': 0.2222222222222222, 'Frio': 0.3333333333333333, 'Moderado': 0.4444444444444444}
{'Quente': 0.4, 'Frio': 0.2, 'Moderado': 0.4}


In [17]:
## E se eu levar em consideração apenas o vento?

print(prob_sim['Vento'])
print(prob_nao['Vento'])

{'VERDADEIRO': 0.3333333333333333, 'FALSO': 0.6666666666666666}
{'VERDADEIRO': 0.6, 'FALSO': 0.4}


# **Classificação de Espécies de Íris com Naive Bayes**

Objetivo:
Neste exercício, você terá a oportunidade de aplicar o algoritmo Naive Bayes para construir um modelo de classificação capaz de categorizar diferentes espécies de íris com base em suas medidas. Este é um passo importante na jornada de aprendizado de máquina, onde você ganhará experiência prática na construção e avaliação de modelos de classificação.

Passos do Exercício:

**1. Aquisição de Dados:**

Você receberá o conjunto de dados "Iris dataset," que contém informações sobre as medidas das sépalas e pétalas de várias espécies de íris.
Este conjunto de dados está prontamente disponível em bibliotecas de machine learning, como o scikit-learn, ou em repositórios online.

**2. Exploração de Dados:**

Comece explorando o conjunto de dados para compreender suas características. Verifique as colunas disponíveis, os tipos de medidas e o número de amostras em cada classe.
Isso ajudará você a ganhar insights iniciais sobre o problema que está sendo abordado.

**3. Pré-processamento de Dados:**

Realize o pré-processamento dos dados, que pode incluir a divisão dos dados em conjuntos de treinamento e teste, normalização ou padronização de recursos (se necessário) e qualquer limpeza de dados relevante.

**4. Treinamento do Modelo Naive Bayes:**

Aprenda como o algoritmo Naive Bayes funciona e como ele pode ser aplicado à classificação de dados numéricos.
Treine um modelo Naive Bayes usando os dados de treinamento que você pré-processou anteriormente.

**5. Avaliação do Modelo:**

Avalie o desempenho do modelo usando métricas de classificação, como precisão, recall, F1-score e matriz de confusão.
Compreenda como essas métricas refletem a capacidade do modelo de fazer previsões precisas.

**6. Tuning de Hiperparâmetros:**

Experimente diferentes configurações de hiperparâmetros do modelo Naive Bayes, como o tipo de distribuição (Gaussiana, Bernoulli, Multinomial), e observe como isso afeta o desempenho.
Isso ajudará você a ajustar o modelo para obter os melhores resultados.

**7. Comparação de Modelos:**

Compare o desempenho do Naive Bayes com outros algoritmos de classificação, como KNN.
Compreenda as forças e fraquezas de cada abordagem.



Este exercício prático é uma oportunidade valiosa para aplicar o algoritmo Naive Bayes a dados numéricos reais e desenvolver suas habilidades em aprendizado de máquina.

In [20]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

iris = load_iris()
X = iris.data  # Matriz de características (medidas)
y = iris.target  # Vetor de rótulos (espécies)

print("Nomes das características: ", iris.feature_names)  # Nomes das características
print("Nomes das espécies: ", iris.target_names)  # Nomes das espécies

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = GaussianNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Acurácia: {accuracy:.2f}')

Nomes das características:  ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Nomes das espécies:  ['setosa' 'versicolor' 'virginica']
Acurácia: 0.98
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.92      0.96        13
           2       0.93      1.00      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.97      0.97        45
weighted avg       0.98      0.98      0.98        45

[[19  0  0]
 [ 0 12  1]
 [ 0  0 13]]


In [22]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

iris = load_iris()
X = iris.data  # Matriz de características (medidas)
y = iris.target  # Vetor de rótulos (espécies)

print("Nomes das características: ", iris.feature_names)  # Nomes das características
print("Nomes das espécies: ", iris.target_names)  # Nomes das espécies

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Acurácia: {accuracy:.2f}')

Nomes das características:  ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Nomes das espécies:  ['setosa' 'versicolor' 'virginica']
Acurácia: 1.00
